#### 불균형 클래스
#### 클래스 불균형에 대한 이해
#### 클래스 가중치를 사용하는 방법
#### 리셈플링 기법
#### 적절한 평가지표

In [24]:
# 불균형데이터 생성 (1:9 비율) 악성0 : 양성1
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import numpy as np

np.random.seed(43) # 값을 고정 (?)

data = load_breast_cancer()
X , y = data.data , data.target

# 악성을 소수 클래스로 생성

print(f'악성 양성의 오리지널 비율 : {np.unique(y,return_counts=True)}')
m_index = np.where(y==0)[0] #악성index # where는 필터링하여 index를 반환하는 것. index 위치에 해당하는 x도 똑같이 가져와야하기 떄문에 index 번호를 알아야해서.where
b_index = np.where(y==1)[0] #양성index

# 악성은 일부만 사용할꺼고 양성은 더 많이 사용할꺼임.
# 악성의 30% 만 사용 , 양성은 전체 1.5 : 8.5
size_30 = int(len(m_index)*0.3)
select_m_index = np.random.choice(m_index , size=size_30 , replace=False)
select_b_index = b_index
# len(select_b_index) / len(select_m_index) + len(select_b_index) # 362.666

concatenate_select_index = np.concatenate( [select_m_index , select_b_index] ) #2개를 합칠때 concatenate()
np.random.shuffle(concatenate_select_index) # concatenate_select_index 자체를 랜덤하게 섞는것. ( 섞어서 반환하는거 아님;)

X_imb = X[concatenate_select_index]
y_imb = y[concatenate_select_index]

# 클래스 분포 확인
unique , counts = np.unique(y_imb,return_counts=True)
print('클래스 분포 : ')
for label , count in zip(unique , counts):
    percentage = count / len(y_imb)*100
    print(f'클래스 {label} : {count}개 ({percentage:.1f}%)')



# 불균형인 상태로 진행
# 스케일링 정규화 StandardScaler
# LogisticRegression
# pipe
# 평가는 class report

x_train, x_test , y_train , y_test = train_test_split(X_imb , y_imb , test_size=0.2 , random_state=42, stratify=y_imb)
print(f'1. 기본모델(불균형 무시함)')
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', LogisticRegression(random_state=42 , max_iter=1000))
])
pipe.fit(x_train, y_train)
print(classification_report(y_test , pipe.predict(x_test),target_names=['악성 0','양성 1']))







악성 양성의 오리지널 비율 : (array([0, 1]), array([212, 357]))
클래스 분포 : 
클래스 0 : 63개 (15.0%)
클래스 1 : 357개 (85.0%)
1. 기본모델(불균형 무시함)
              precision    recall  f1-score   support

        악성 0       1.00      0.85      0.92        13
        양성 1       0.97      1.00      0.99        71

    accuracy                           0.98        84
   macro avg       0.99      0.92      0.95        84
weighted avg       0.98      0.98      0.98        84



In [25]:
print('불균형 해결 : 클래스 가중치 사용')

# 기존 파이프라인의 clf 이름의 객체의 파라메터를 조정

from copy import deepcopy

pipe_weight = deepcopy(pipe)
pipe_weight.set_params(clf__class_weight = 'balanced') # 언더바언더바?
pipe_weight.fit(x_train ,  y_train)
print( classification_report(y_test , pipe_weight.predict(x_test),target_names=['악성 0','양성 1'])) 

print('가중치 계산')
n_sample = len(y_train)
n_classes = 2
class_counts = np.bincount(y_train)
for i in range(n_classes):
    weight = n_sample / (n_classes * class_counts[i])
    print(f'클래스 : {i} : {weight:.3f}')




불균형 해결 : 클래스 가중치 사용
              precision    recall  f1-score   support

        악성 0       0.86      0.92      0.89        13
        양성 1       0.99      0.97      0.98        71

    accuracy                           0.96        84
   macro avg       0.92      0.95      0.93        84
weighted avg       0.97      0.96      0.96        84

가중치 계산
클래스 : 0 : 3.360
클래스 : 1 : 0.587


In [ ]:
# RandomForest 균형모드로 만들기.
print('불균형 : RandomForest (균형모드)')
pipe_rf = Pipeline([
    ('scaler',StandardScaler()),
    ('clf', RandomForestClassifier(class_weight='balanced' , random_state=43 , n_estimators=100)) # n_estimators=100개가지
])
# 랜덤포레스트에서 class_weight = 'balanced' 가중치 제거하면 score가 올라감 
pipe_rf.fit(x_train,y_train)
y_pred = pipe_rf.predict(x_test)
print( classification_report(y_test , y_pred))


불균형 : RandomForest (균형모드)
              precision    recall  f1-score   support

           0       0.79      0.85      0.81        13
           1       0.97      0.96      0.96        71

    accuracy                           0.94        84
   macro avg       0.88      0.90      0.89        84
weighted avg       0.94      0.94      0.94        84



In [ ]:
# oversampling
# undersampling
# SMOT , SMOTEEN
# 과적합을 잡아보세요.